## Install Dependencies

In [11]:
# %pip install earthengine-api -q
# %pip install git+https://github.com/cordmaur/GEES2Downloader.git@main -q
# %pip install geemap -q

In [12]:
# %pip install pympler

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import ee
import matplotlib.pyplot as plt
from pathlib import Path

import xarray as xr
import rasterio as rio

from waterdetect.planetary import search_tiles
from waterdetect.engine import WaterDetect
from waterdetect.cloudless import get_gee_img

In [15]:
# ee.Authenticate()
ee.Initialize()

In [16]:
# For memory tracking
from IPython.display import clear_output
from pympler import asizeof, muppy, summary


# Getting the Images to be Processed

In [17]:
from pathlib import Path

tile = '22KDA'
period = '2018-12/2021-12'
output_folder = 'E:/output/planetary'

# Create the output folder (tile name)
out_path = Path(output_folder)/tile
if not out_path.exists():
    out_path.mkdir(exist_ok=True)

imgs = search_tiles(tile, period, reverse=False)
len(imgs)

366

## Create Logger

In [18]:
import logging

logger = logging.getLogger('waterdetect-1')
handler = logging.FileHandler(f'./tmp/log_{tile}.txt', mode='a+')
handler.setFormatter(logging.Formatter("%(asctime)-15s %(levelname)-8s %(message)s"))
logger.setLevel(logging.INFO)
logger.addHandler(handler)

## Process the Images

In [19]:
import gc

def process_img(img, out_folder, n_jobs=7, retries=3):
    """
    Fully waterdetect one img (stac item) and save the outputs to the out_folder.
    If something goes wrong in the meantime, it will retry the execution.
    """
    
    print('*'*50)
    print(f'Processing tile {img}')
    
    while retries > 0:
        try:
            wd = WaterDetect(img, cluster_bands, s2clouds=True, n_jobs=n_jobs)
            wd.run_detect_water()

            # save thumbnails
            wd.plot_thumbs(cols=3, thumbs=['rgb', 'watermask', 'mndwi', 'mask', 'glint', 'ndwi'],  save_folder=out_folder)

            # save graphs
            wd.plot_graphs([['ndwi', 'mndwi'], ['B12', 'mndwi']], cols=2, save_folder=out_folder)

            # save the watermask
            wd.save_geotiff('nodata_watermask', out_folder/f'{wd.img_item.id[:38]}_watermask.tif')

            retries = 0
            logger.info(f'{img} - OK')

        except Exception as e:
            
            logger.error(f'Exception {e} when processing img {img}')
            logger.error(f'Releasing memory')
            
            
            retries -= 1
            logger.error(f'There are {retries} retries left!')

        # Regardless the result of the processing, will release the memory by killing wd
        try:
            del wd
        except:
            pass
        
        # And collect the garbage
        gc.collect()

            

In [ ]:
from datetime import datetime

cluster_bands = ['mndwi', 'ndwi', 'B12']

lap = datetime.now()

for img in imgs:
    clear_output(wait=True)
    
    print(datetime.now() - lap)
    lap = datetime.now()
    
    sum1 = summary.summarize(muppy.get_objects())
    summary.print_(sum1)

    process_img(img, out_path)
        

0:01:59.426805
                           types |   # objects |   total size
================================ | =========== | ============
                   numpy.ndarray |        6246 |      2.81 GB
                            dict |      374482 |    108.49 MB
                             str |      342882 |     48.92 MB
                            list |      206435 |     21.25 MB
                            code |       50005 |      8.97 MB
                            type |        7538 |      6.74 MB
                           tuple |       96438 |      5.30 MB
  traitlets.config.loader.Config |       16404 |      3.88 MB
                           float |      161050 |      3.69 MB
                             int |      129300 |      3.51 MB
                             set |        5747 |      2.59 MB
            asttokens.util.Token |       20482 |      2.19 MB
                           bytes |       16703 |      1.11 MB
                          method |       17724 |      1

Tiles:   0%|          | 0/4 [00:00<?, ?tile/s]

Tile[0:2896,0:2896]:   0%|          | 0.00/10.9M [00:00<?, ?b/s]

Tile[0:2896,2896:5490]:   0%|          | 0.00/9.77M [00:00<?, ?b/s]

Tile[2896:5490,0:2896]:   0%|          | 0.00/9.77M [00:00<?, ?b/s]

Tile[2896:5490,2896:5490]:   0%|          | 0.00/8.75M [00:00<?, ?b/s]

Finished. The result can be accessed at obj.array
Getting metadata: granule-metadata
